## Building Fashion Recommendation Systems
- Businesses such as Spotify and Netflix have been trailblazers in the recommendation systems world. More than 80% of the TV shows people watch on Netflix are discovered through the platform’s recommendation system (Wired) and Discover Weekly playlists boasted 40 million unique users just a year after it launched in July 2015 (TechCrunch).

- In contrast, the fashion industry has been relatively slow on the uptake when it comes to robust recommendation systems, but for good reason.

## Method to build Fashion Recommmeder System
#### Basically our model is based on the method of reverse image search.
For example : If an user upload a image of product on site , then our model recommends 5 more products related to that product by comapring the feature of that product with other.

#### we are going to use transferlearning(Resnet50) technique for extracting the feature of product
- Step 1 - We are going to import the resnet50 model
- Step2 - We are going to extract the features of products with the help of resnet50
- Step3 - Export the features (we are going to save that extracted features somewhere(like list) to repeatedly use them)
- Step 4 - We are going to recommnend the products related to the uploaded product..(with the help of Nearest Neighbour Algorithm)


## Lets import all the required Libraries

In [1]:
import tensorflow
from tensorflow.keras.preprocessing import image ## for loading the image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np
from numpy.linalg import norm ## for naormalizing the features
import os
from tqdm import tqdm ## for checking the process of the for loop.
import pickle

## Initializing the model

In [2]:
model=ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

#### As we know resnet50 is already trained on imagenet dataset with very good accuracy, so we don't need to train it again.

In [3]:
model.trainable=False

## Building the model

In [4]:
model=tensorflow.keras.Sequential([model,GlobalMaxPooling2D()])

In [5]:
## Lets see our model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_max_pooling2d (Global (None, 2048)              0         
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________


#### we can see that resnet50 returns 2048 features of one image

## Now we are going to extract the features from resnet50

In [6]:
def extract_features(image_path,model):
    ## we have to load the image first
    img=image.load_img(image_path,target_size=(224,224))
    ## Converting the image into array
    image_array=image.img_to_array(img)
    ## Now expanding the dimension of image (as we know the image with batch size is only accepted by resnet50)
    expanded_img=np.expand_dims(image_array,axis=0)
    ## Now we are going to preprocess the image
    img_preprocessed=preprocess_input(expanded_img)
    ## Now we are going to take predictions from the mode;
    predictions=model.predict(img_preprocessed).flatten()
    ## Now we are going to normalize the image (Just to scale down the image)
    prediction_normalized= predictions/norm(predictions)
    return prediction_normalized

#### Now we have to store the all the image path in one list

In [7]:
print(len(os.listdir('images')))

44441


#### we can see that we have 44k images in our datastet

In [8]:
file = []
for i in os.listdir('images'):
    file.append(os.path.join('images',i))

#### we have all the paths of our iamges in our file list

#### Now we have to pass image_path one by one to our extract_feature method to extract the feature and going to save these features in a list


In [9]:
features_list=[]
for i in tqdm(file):
    features_list.append(extract_features(i,model))

100%|██████████████████████████████████████████████████████████████████████████| 44441/44441 [3:31:16<00:00,  3.51it/s]


In [10]:
pickle.dump(features_list,open('feature.pkl','wb'))
pickle.dump(file,open('file_name.pkl','wb'))